In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/carboni/Downloads/Leopard.csv')

### Clusters and other keywords

In [3]:
keyword = 'Bête'
df = df[df['Cluster'].str.contains(keyword, case=False, na=False)]

In [4]:
df.head()

,Cluster,manifest_url,canvas_number,image_url,City,Country,Title,wkt,Date,Journal Type,Unnamed: 10
0,Bête Seule (Tigre),https://iiif.unige.ch/dhportal/ug796777/manifest,26.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2019-01-01,Geography,NaN
4,Bête et Corps Léopard,https://digi.ub.uni-heidelberg.de/diglit/iiif/...,54.0,https://digi.ub.uni-heidelberg.de/iiif/2/porta...,Gdańsk,Poland,Porta aurea : rocznik Instytutu Historii Sztuk...,POINT(18.655 54.348055555555554),2018-01-01,Art History,NaN
5,Bête Seule (Tigre),https://iiif.unige.ch/dhportal/ug622322/manifest,40.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2018-01-01,Geography,NaN
9,Bête Seule Léopard,https://iiif.unige.ch/dhportal/ug673896/manifest,115.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2017-01-01,Geography,NaN
10,Bête Seule Léopard,https://iiif.unige.ch/dhportal/ug673896/manifest,116.0,https://iiif.unige.ch/iiif/2/visualcontagions/...,"Washington, D.C.",United States of America,National Geographic Society,POINT(-77.036666666667 38.895),2017-01-01,Geography,NaN


## Index city 

### If WKT

In [7]:
df[['lon', 'lat']] = df['wkt'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

### if not wkt

In [8]:
city_coords = df.groupby('City').agg({'lat': 'first', 'lon': 'first'}).reset_index()

In [9]:
city_coords.rename({'City': 'name'}, axis=1, inplace=True)

In [10]:
city_coords.index = city_coords.index + 1
city_coords.index.name = 'id'

In [11]:
print(city_coords)

                 name                lat                  lon
id                                                           
1            Aberdeen  46.97536111111111  -123.81572222222222
2         Albuquerque    35.116666666667     -106.61666666667
3             Algiers    36.776388888889      3.0586111111111
4                Alma    43.378888888889     -84.659722222222
5            Amarillo    35.199166666667     -101.84527777778
..                ...                ...                  ...
182           Wichita    37.688888888889     -97.336111111111
183        Wilmington    34.223333333333     -77.912222222222
184  Wisconsin Rapids    44.394444444444     -89.832777777778
185              Yale    43.130277777778     -82.797777777778
186            Évreux    49.023333333333               1.1525

[186 rows x 3 columns]


In [12]:
city_coords.to_csv('od_city_index.csv', index=True)

## Index to OD Matrix

In [13]:
city_id_map = city_coords.reset_index().set_index('name')['id'].to_dict()

In [14]:
df['origin'] = df['City'].map(city_id_map).astype(int)
df['dest'] = df['City'].shift(-1).map(city_id_map).fillna(method='ffill').astype(int)
df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

In [15]:
#df.iloc[-1, df.columns.get_loc('dest')] = df.iloc[-1]['origin']

In [16]:
od_matrix = df.groupby(['origin', 'dest']).size().reset_index(name='count')

In [17]:
print(od_matrix)

     origin  dest  count
0         1   103      2
1         2     2      1
2         2    36      1
3         2    57      1
4         2   129      1
..      ...   ...    ...
614     182   103      1
615     183    62      1
616     184    12      1
617     185   103      1
618     186   133      1

[619 rows x 3 columns]


In [18]:
od_matrix.to_csv('od_matrix.csv', index=True)